In [1]:
from calendar_planner import Calendar

In [2]:
# path to files
course_paths = {
    "DB": "excel_files/db_inf_ik.xlsx",
    "IKI": "excel_files/iki.xlsx",
    "NWO": "excel_files/netwerk.xlsx",
    "PSAS": "excel_files/probsearch.xlsx"
}

# configuration
lecture_types = ["hoorcollege"]
practical_types = ["laptopcollege", "werkcollege", "presentatie"]
ignore = []
ignore_description = ["optional", "reistijd"]

# create calendar 
calendar = Calendar(
    lecture_types=lecture_types, 
    practical_types=practical_types,
    ignore=ignore,
    ignore_description=ignore_description
)
calendar.read_courses_from_excel(course_paths)

print(calendar)

Calendar contains: DB, IKI, NWO, PSAS and 0 other event(s)


In [3]:
# get all possible combinations
# Parameters
#     - format_groups = simplifies output
#     - start_time = lower time limit
#     - end_time = upper time limit
# time limit is for searching combinations,
# such that no lecture is before [start_time]
# and no lecture is after [end_time]
df = calendar.find_all_schedules(
    format_groups=True, 
    start_time="10:00", 
    end_time="17:00"
)
df.head(10)

,DB,IKI,NWO,PSAS
77,I,K,D,E
78,I,K,D,A
79,I,K,D,D
80,I,K,D,B
81,I,K,D,G
82,I,K,D,C
83,I,K,D,F
91,H,K,D,E
92,H,K,D,A
93,H,K,D,D
